In [ ]:
import time
import numpy as np
from sentence_transformers import SentenceTransformer, util
from pathlib import Path
import json
from typing import List, Dict, Tuple
import pandas as pd
from dataclasses import dataclass

# ============================================================================
# RECOMMENDED EMBEDDING MODELS FOR CODE DOCUMENTATION
# ============================================================================

EMBEDDING_MODELS = {
    # === BEST FOR CODE (RECOMMENDED) ===
    "codebert": {
        "name": "microsoft/codebert-base",
        "size_mb": 500,
        "dims": 768,
        "description": "Trained on code + NL, excellent for code docs",
        "best_for": "Code-text hybrid content"
    },
    
    "unixcoder": {
        "name": "microsoft/unixcoder-base",
        "size_mb": 500,
        "dims": 768,
        "description": "Cross-lingual code model, very good for technical docs",
        "best_for": "API documentation and code snippets"
    },
    
    # === GENERAL PURPOSE (GOOD BALANCE) ===
    "bge-small": {
        "name": "BAAI/bge-small-en-v1.5",
        "size_mb": 134,
        "dims": 384,
        "description": "Fast, lightweight, good for general text",
        "best_for": "Speed and memory efficiency"
    },
    
    "bge-base": {
        "name": "BAAI/bge-base-en-v1.5",
        "size_mb": 438,
        "dims": 768,
        "description": "Balanced performance and speed",
        "best_for": "Production use (best balance)"
    },
    
    "bge-large": {
        "name": "BAAI/bge-large-en-v1.5",
        "size_mb": 1340,
        "dims": 1024,
        "description": "Highest quality, slower",
        "best_for": "Maximum accuracy"
    },
    
    # === CURRENT DEFAULT ===
    "minilm": {
        "name": "sentence-transformers/all-MiniLM-L6-v2",
        "size_mb": 90,
        "dims": 384,
        "description": "Very fast, lightweight, your current model",
        "best_for": "Baseline comparison"
    },
    
    # === SPECIALIZED ===
    "instructor": {
        "name": "hkunlp/instructor-base",
        "size_mb": 440,
        "dims": 768,
        "description": "Instruction-aware embeddings",
        "best_for": "Task-specific embeddings"
    },
    
    "e5-small": {
        "name": "intfloat/e5-small-v2",
        "size_mb": 134,
        "dims": 384,
        "description": "Microsoft's E5, strong performance",
        "best_for": "Good general-purpose alternative"
    },
    
    "e5-base": {
        "name": "intfloat/e5-base-v2",
        "size_mb": 438,
        "dims": 768,
        "description": "Microsoft's E5, better accuracy",
        "best_for": "Higher quality than e5-small"
    },
}

# ============================================================================
# EVALUATION FRAMEWORK
# ============================================================================

@dataclass
class ModelMetrics:
    """Metrics for a single model"""
    model_name: str
    model_id: str
    
    # Performance metrics
    mrr: float  # Mean Reciprocal Rank
    recall_at_3: float
    recall_at_5: float
    ndcg_at_5: float  # Normalized Discounted Cumulative Gain
    
    # Efficiency metrics
    encoding_time_per_chunk: float  # seconds
    search_time_per_query: float  # seconds
    model_size_mb: float
    embedding_dims: int
    
    # Quality metrics
    avg_positive_similarity: float
    avg_negative_similarity: float
    separation_score: float  # positive_sim - negative_sim
    
    def to_dict(self) -> Dict:
        return {
            'model_name': self.model_name,
            'model_id': self.model_id,
            'mrr': f"{self.mrr:.4f}",
            'recall@3': f"{self.recall_at_3:.4f}",
            'recall@5': f"{self.recall_at_5:.4f}",
            'ndcg@5': f"{self.ndcg_at_5:.4f}",
            'encode_time_ms': f"{self.encoding_time_per_chunk * 1000:.2f}",
            'search_time_ms': f"{self.search_time_per_query * 1000:.2f}",
            'size_mb': f"{self.model_size_mb:.1f}",
            'dims': self.embedding_dims,
            'pos_sim': f"{self.avg_positive_similarity:.4f}",
            'neg_sim': f"{self.avg_negative_similarity:.4f}",
            'separation': f"{self.separation_score:.4f}"
        }


class EmbeddingModelComparison:
    """Compare multiple embedding models for code documentation"""
    
    def __init__(self, chunks: List[Dict], test_queries: List[Dict] = None):
        """
        Initialize comparison framework
        
        Args:
            chunks: List of documentation chunks
            test_queries: List of test queries with ground truth
                         Format: [{"query": "...", "relevant_ids": [...]}, ...]
        """
        self.chunks = chunks
        self.test_queries = test_queries or self._generate_test_queries()
        self.results = {}
        
    def _generate_test_queries(self) -> List[Dict]:
        """Generate test queries from chunks"""
        print("🔍 Generating test queries from chunks...")
        
        queries = []
        
        # Sample diverse chunks
        sampled_chunks = np.random.choice(
            self.chunks, 
            min(50, len(self.chunks)), 
            replace=False
        )
        
        for chunk in sampled_chunks:
            # Create query from chunk context
            context = chunk.get('context', '')
            text = chunk.get('text', '')
            chunk_id = chunk.get('metadata', {}).get('id', '')
            
            # Extract key phrases for query
            if context:
                # Use first sentence of context as query
                query = context.split('.')[0].strip()
                if len(query) > 10:  # Valid query
                    queries.append({
                        'query': query,
                        'relevant_ids': [chunk_id],
                        'source': 'context'
                    })
            
            # Create queries from code examples
            if 'code' in text.lower() or 'def ' in text:
                # Extract function names or key terms
                words = text.split()
                if len(words) >= 5:
                    query = ' '.join(words[:8])
                    queries.append({
                        'query': query,
                        'relevant_ids': [chunk_id],
                        'source': 'code'
                    })
        
        print(f"✅ Generated {len(queries)} test queries")
        return queries[:30]  # Limit to 30 queries
    
    def evaluate_model(self, model_name: str, model_id: str) -> ModelMetrics:
        """
        Evaluate a single embedding model
        
        Args:
            model_name: Display name for the model
            model_id: HuggingFace model ID
            
        Returns:
            ModelMetrics with all evaluation scores
        """
        print(f"\n{'='*80}")
        print(f"EVALUATING: {model_name}")
        print(f"Model ID: {model_id}")
        print(f"{'='*80}")
        
        # Load model
        print("📥 Loading model...")
        start_load = time.time()
        try:
            model = SentenceTransformer(model_id)
        except Exception as e:
            print(f"❌ Failed to load model: {e}")
            return None
        
        load_time = time.time() - start_load
        print(f"✅ Model loaded in {load_time:.2f}s")
        
        # Get model info
        embedding_dims = model.get_sentence_embedding_dimension()
        model_size_mb = EMBEDDING_MODELS.get(model_name.lower(), {}).get('size_mb', 0)
        
        # Encode chunks
        print(f"\n📊 Encoding {len(self.chunks)} chunks...")
        chunk_texts = [f"{c['context']}\n{c['text']}" for c in self.chunks]
        
        start_encode = time.time()
        chunk_embeddings = model.encode(
            chunk_texts, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        encoding_time = time.time() - start_encode
        encoding_time_per_chunk = encoding_time / len(self.chunks)
        
        print(f"✅ Encoding completed in {encoding_time:.2f}s")
        print(f"   ({encoding_time_per_chunk*1000:.2f}ms per chunk)")
        
        # Evaluate search quality
        print(f"\n🔍 Evaluating search quality with {len(self.test_queries)} queries...")
        
        mrr_scores = []
        recall_at_3_scores = []
        recall_at_5_scores = []
        ndcg_scores = []
        search_times = []
        
        for query_data in self.test_queries:
            query = query_data['query']
            relevant_ids = set(query_data['relevant_ids'])
            
            # Encode query and search
            start_search = time.time()
            query_embedding = model.encode([query], convert_to_numpy=True)
            
            # Compute similarities
            similarities = util.cos_sim(query_embedding, chunk_embeddings)[0]
            top_indices = np.argsort(-similarities.numpy())[:5]
            
            search_time = time.time() - start_search
            search_times.append(search_time)
            
            # Get top result IDs
            top_ids = [
                self.chunks[idx].get('metadata', {}).get('id', '')
                for idx in top_indices
            ]
            
            # Calculate metrics
            # MRR (Mean Reciprocal Rank)
            for rank, chunk_id in enumerate(top_ids, 1):
                if chunk_id in relevant_ids:
                    mrr_scores.append(1.0 / rank)
                    break
            else:
                mrr_scores.append(0.0)
            
            # Recall@3 and Recall@5
            recall_at_3 = len(set(top_ids[:3]) & relevant_ids) / len(relevant_ids)
            recall_at_5 = len(set(top_ids[:5]) & relevant_ids) / len(relevant_ids)
            recall_at_3_scores.append(recall_at_3)
            recall_at_5_scores.append(recall_at_5)
            
            # NDCG@5 (simplified)
            dcg = sum([
                1.0 / np.log2(rank + 1) 
                for rank, chunk_id in enumerate(top_ids, 1) 
                if chunk_id in relevant_ids
            ])
            idcg = sum([1.0 / np.log2(i + 1) for i in range(1, min(6, len(relevant_ids) + 1))])
            ndcg = dcg / idcg if idcg > 0 else 0.0
            ndcg_scores.append(ndcg)
        
        # Evaluate embedding quality (similarity distribution)
        print(f"\n📈 Evaluating embedding quality...")
        
        # Sample pairs for similarity analysis
        num_samples = min(100, len(self.chunks))
        sampled_indices = np.random.choice(len(self.chunks), num_samples, replace=False)
        sampled_embeddings = chunk_embeddings[sampled_indices]
        
        # Compute similarity matrix
        similarity_matrix = util.cos_sim(sampled_embeddings, sampled_embeddings)
        
        # Calculate positive similarities (same document)
        positive_sims = []
        negative_sims = []
        
        for i in range(len(sampled_indices)):
            for j in range(i + 1, len(sampled_indices)):
                sim = similarity_matrix[i][j].item()
                
                # Check if same document
                doc_i = self.chunks[sampled_indices[i]].get('metadata', {}).get('id', '')
                doc_j = self.chunks[sampled_indices[j]].get('metadata', {}).get('id', '')
                
                if doc_i == doc_j and doc_i:
                    positive_sims.append(sim)
                else:
                    negative_sims.append(sim)
        
        avg_positive_sim = np.mean(positive_sims) if positive_sims else 0.0
        avg_negative_sim = np.mean(negative_sims) if negative_sims else 0.0
        separation_score = avg_positive_sim - avg_negative_sim
        
        # Create metrics
        metrics = ModelMetrics(
            model_name=model_name,
            model_id=model_id,
            mrr=np.mean(mrr_scores),
            recall_at_3=np.mean(recall_at_3_scores),
            recall_at_5=np.mean(recall_at_5_scores),
            ndcg_at_5=np.mean(ndcg_scores),
            encoding_time_per_chunk=encoding_time_per_chunk,
            search_time_per_query=np.mean(search_times),
            model_size_mb=model_size_mb,
            embedding_dims=embedding_dims,
            avg_positive_similarity=avg_positive_sim,
            avg_negative_similarity=avg_negative_sim,
            separation_score=separation_score
        )
        
        # Print summary
        print(f"\n📊 RESULTS FOR {model_name}:")
        print(f"   MRR: {metrics.mrr:.4f}")
        print(f"   Recall@3: {metrics.recall_at_3:.4f}")
        print(f"   Recall@5: {metrics.recall_at_5:.4f}")
        print(f"   NDCG@5: {metrics.ndcg_at_5:.4f}")
        print(f"   Encoding: {metrics.encoding_time_per_chunk*1000:.2f}ms/chunk")
        print(f"   Search: {metrics.search_time_per_query*1000:.2f}ms/query")
        print(f"   Separation: {metrics.separation_score:.4f}")
        
        return metrics
    
    def compare_models(self, model_list: List[str] = None) -> pd.DataFrame:
        """
        Compare multiple models
        
        Args:
            model_list: List of model keys from EMBEDDING_MODELS
                       If None, compares recommended models
        """
        if model_list is None:
            # Default: Compare best models for code
            model_list = ["bge-base", "bge-small", "minilm", "e5-base"]
        
        results = []
        
        for model_key in model_list:
            if model_key not in EMBEDDING_MODELS:
                print(f"⚠️ Unknown model: {model_key}")
                continue
            
            model_info = EMBEDDING_MODELS[model_key]
            
            try:
                metrics = self.evaluate_model(
                    model_name=model_key,
                    model_id=model_info['name']
                )
                
                if metrics:
                    results.append(metrics.to_dict())
                    self.results[model_key] = metrics
                    
            except Exception as e:
                print(f"❌ Error evaluating {model_key}: {e}")
                continue
        
        # Create comparison DataFrame
        df = pd.DataFrame(results)
        
        # Print comparison table
        print("\n" + "="*80)
        print("FINAL COMPARISON")
        print("="*80)
        print(df.to_string(index=False))
        
        # Save results
        output_path = Path("embedding_comparison_results.json")
        with open(output_path, 'w') as f:
            json.dump(results, f, indent=2)
        print(f"\n💾 Results saved to {output_path}")
        
        return df
    
    def get_recommendation(self) -> str:
        """Get model recommendation based on results"""
        if not self.results:
            return "No results available. Run compare_models() first."
        
        print("\n" + "="*80)
        print("RECOMMENDATIONS")
        print("="*80)
        
        # Find best models for different criteria
        best_accuracy = max(self.results.items(), key=lambda x: x[1].mrr)
        best_speed = min(self.results.items(), key=lambda x: x[1].encoding_time_per_chunk)
        best_balance = max(self.results.items(), 
                          key=lambda x: x[1].mrr / (x[1].encoding_time_per_chunk * 1000 + 1))
        
        print(f"\n🎯 BEST ACCURACY: {best_accuracy[0]}")
        print(f"   MRR: {best_accuracy[1].mrr:.4f}")
        print(f"   Size: {best_accuracy[1].model_size_mb:.1f}MB")
        
        print(f"\n⚡ FASTEST: {best_speed[0]}")
        print(f"   Speed: {best_speed[1].encoding_time_per_chunk*1000:.2f}ms/chunk")
        print(f"   MRR: {best_speed[1].mrr:.4f}")
        
        print(f"\n⚖️  BEST BALANCE: {best_balance[0]}")
        print(f"   MRR: {best_balance[1].mrr:.4f}")
        print(f"   Speed: {best_balance[1].encoding_time_per_chunk*1000:.2f}ms/chunk")
        print(f"   Size: {best_balance[1].model_size_mb:.1f}MB")
        
        return best_balance[0]
    
    def save_best_model(self, criterion: str = "balance", output_dir: str = "models/best"):
        """
        Save the best model based on specified criterion
        
        Args:
            criterion: "balance", "accuracy", or "speed"
            output_dir: Directory to save the best model
            
        Returns:
            Path to saved model
        """
        if not self.results:
            print("❌ No results available. Run compare_models() first.")
            return None
        
        print("\n" + "="*80)
        print(f"SAVING BEST MODEL (criterion: {criterion})")
        print("="*80)
        
        # Select best model based on criterion
        if criterion == "accuracy":
            best_model_key, best_metrics = max(
                self.results.items(), 
                key=lambda x: x[1].mrr
            )
            criterion_name = "Accuracy (MRR)"
        elif criterion == "speed":
            best_model_key, best_metrics = min(
                self.results.items(), 
                key=lambda x: x[1].encoding_time_per_chunk
            )
            criterion_name = "Speed"
        else:  # balance
            best_model_key, best_metrics = max(
                self.results.items(), 
                key=lambda x: x[1].mrr / (x[1].encoding_time_per_chunk * 1000 + 1)
            )
            criterion_name = "Balance (MRR/Speed)"
        
        print(f"\n🏆 BEST MODEL BY {criterion_name}: {best_model_key}")
        print(f"   Model ID: {best_metrics.model_id}")
        print(f"   MRR: {best_metrics.mrr:.4f}")
        print(f"   Speed: {best_metrics.encoding_time_per_chunk*1000:.2f}ms/chunk")
        print(f"   Size: {best_metrics.model_size_mb:.1f}MB")
        print(f"   Dimensions: {best_metrics.embedding_dims}")
        
        # Create output directory
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        # Load and save the best model
        print(f"\n📥 Loading best model...")
        try:
            best_model = SentenceTransformer(best_metrics.model_id)
            
            # Save model
            model_save_path = output_path / "embedding_model"
            best_model.save(str(model_save_path))
            print(f"✅ Model saved to: {model_save_path}")
            
            # Save metadata
            metadata = {
                "model_name": best_model_key,
                "model_id": best_metrics.model_id,
                "selection_criterion": criterion,
                "metrics": best_metrics.to_dict(),
                "description": EMBEDDING_MODELS.get(best_model_key, {}).get('description', ''),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }
            
            metadata_path = output_path / "model_info.json"
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=2)
            print(f"✅ Metadata saved to: {metadata_path}")
            
            # Save comparison results
            comparison_path = output_path / "comparison_results.json"
            all_results = [metrics.to_dict() for metrics in self.results.values()]
            with open(comparison_path, 'w') as f:
                json.dump(all_results, f, indent=2)
            print(f"✅ Comparison results saved to: {comparison_path}")
            
            print(f"\n" + "="*80)
            print(f"✅ BEST MODEL SAVED SUCCESSFULLY")
            print(f"="*80)
            print(f"\n📂 Output directory: {output_path.absolute()}")
            print(f"\nTo use this model in your RAG system:")
            print(f"   config.base_model = '{best_metrics.model_id}'")
            print(f"\nOr load from saved path:")
            print(f"   model = SentenceTransformer('{model_save_path.absolute()}')")
            
            return model_save_path
            
        except Exception as e:
            print(f"❌ Error saving model: {e}")
            import traceback
            traceback.print_exc()
            return None


# ============================================================================
# USAGE EXAMPLE
# ============================================================================

def run_model_comparison(chunks: List[Dict], save_best: bool = True):
    """
    Run complete model comparison and optionally save best model
    
    Args:
        chunks: Your preprocessed documentation chunks
        save_best: Whether to automatically save the best model
    """
    # Initialize comparison
    comparison = EmbeddingModelComparison(chunks)
    
    # Compare recommended models for code documentation
    print("\n🚀 Starting Model Comparison...")
    print(f"   Testing with {len(chunks)} chunks")
    print(f"   Using {len(comparison.test_queries)} test queries")
    
    # Option 1: Compare specific models
    models_to_test = [
        # "codebert",
        # "unixcoder" 
        # "bge-base",      # Best general-purpose
        # "bge-small",     # Fast and efficient
        "minilm"        # Your current baseline
        # "e5-base",       # Strong alternative
        # "codebert",    # Uncomment if testing code-specific models
    ]
    
    results_df = comparison.compare_models(models_to_test)
    
    # Get recommendation
    best_model = comparison.get_recommendation()
    
    print(f"\n✅ RECOMMENDED MODEL: {best_model}")
    print(f"\n💡 To use this model, update your EmbeddingConfig:")
    print(f"   base_model = '{EMBEDDING_MODELS[best_model]['name']}'")
    
    # Save best model if requested
    if save_best:
        print("\n" + "="*80)
        print("SAVING BEST MODELS")
        print("="*80)
        
        # Save all three variants
        print("\n📦 Saving best model by different criteria...")
        
        # 1. Best balance (recommended)
        balance_path = comparison.save_best_model(
            criterion="balance", 
            output_dir="models/best"
        )
        
        # 2. Best accuracy
        accuracy_path = comparison.save_best_model(
            criterion="accuracy",
            output_dir="models/best_accuracy"
        )
        
        # 3. Best speed
        speed_path = comparison.save_best_model(
            criterion="speed",
            output_dir="models/best_speed"
        )
        
        print("\n" + "="*80)
        print("SAVED MODEL PATHS")
        print("="*80)
        print(f"\n⚖️  Best Balance: {balance_path}")
        print(f"🎯 Best Accuracy: {accuracy_path}")
        print(f"⚡ Best Speed: {speed_path}")
        
        print("\n💡 Recommended for production: Use 'models/best'")
    
    return comparison, results_df


# ============================================================================
# STANDALONE FUNCTION TO SAVE BEST MODEL
# ============================================================================

def save_comparison_best_model(comparison: EmbeddingModelComparison, 
                               criterion: str = "balance",
                               output_dir: str = "models/best"):
    """
    Standalone function to save best model after comparison
    
    Args:
        comparison: EmbeddingModelComparison instance with results
        criterion: "balance", "accuracy", or "speed"
        output_dir: Directory to save model
        
    Returns:
        Path to saved model
    
    Example:
        comparison, results = run_model_comparison(chunks, save_best=False)
        
        # Later, save specific model
        save_comparison_best_model(comparison, criterion="accuracy", 
                                   output_dir="models/best_accuracy")
    """
    return comparison.save_best_model(criterion=criterion, output_dir=output_dir)


# # Example usage:
# if __name__ == "__main__":
#     config = EmbeddingConfig("pyautogui")
    
#     # Step 1: Load and preprocess data
#     print("\n[STEP 1/5] Loading and preprocessing data...")
#     loader = RAGDataLoader(config)
#     df = loader.load_data()
#     chunks = loader.prepare_all_chunks()
#     # Load your chunks
#     # chunks = loader.prepare_all_chunks()
    
#     # Option 1: Run comparison and auto-save best model
#     comparison, results = run_model_comparison(chunks, save_best=True)
    
#     # Option 2: Run comparison without saving, then save later
#     # comparison, results = run_model_comparison(chunks, save_best=False)
#     # 
#     # # Review results, then save manually
#     # save_comparison_best_model(comparison, criterion="balance", output_dir="models/best")
#     # save_comparison_best_model(comparison, criterion="accuracy", output_dir="models/best_accuracy")
#     # save_comparison_best_model(comparison, criterion="speed", output_dir="models/best_speed")
    
    # pass

In [ ]:
# Example usage:
if __name__ == "__main__":
    config = EmbeddingConfig("pyautogui")
    
    # Step 1: Load and preprocess data
    print("\n[STEP 1/5] Loading and preprocessing data...")
    loader = RAGDataLoader(config)
    df = loader.load_data()
    chunks = loader.prepare_all_chunks()
    # Load your chunks
    # chunks = loader.prepare_all_chunks()
    
    # Option 1: Run comparison and auto-save best model
    comparison, results = run_model_comparison(chunks, save_best=False)
    
    # Option 2: Run comparison without saving, then save later
    # comparison, results = run_model_comparison(chunks, save_best=False)
    # 
    # # Review results, then save manually
    save_comparison_best_model(comparison, criterion="balance", output_dir="models/best")
    # save_comparison_best_model(comparison, criterion="accuracy", output_dir="models/best_accuracy")
    # save_comparison_best_model(comparison, criterion="speed", output_dir="models/best_speed")
    

In [ ]:
# STEP 2: Add Top-K Analysis Function (NEW CODE TO ADD)
# ======================================================

def analyze_optimal_topk_for_best_model(comparison: EmbeddingModelComparison, 
                                        chunks: List[Dict],
                                        best_model_name: str = None,
                                        k_values: List[int] = None):
    """
    Analyze optimal top-k for the best performing model
    
    Args:
        comparison: Your existing EmbeddingModelComparison object
        chunks: Your chunk data
        best_model_name: Name of best model (if None, uses best from comparison)
        k_values: K values to test (default: [1,2,3,5,7,10])
    
    Returns:
        Dictionary with optimal k and detailed results
    """
    from sentence_transformers import SentenceTransformer, util
    import numpy as np
    import matplotlib.pyplot as plt
    
    if k_values is None:
        k_values = [1, 2, 3, 5, 7, 10]
    
    # Step 2.1: Identify best model
    if best_model_name is None:
        # Get best model from comparison results
        best_model_name = max(
            comparison.results.items(),
            key=lambda x: x[1].mrr
        )[0]
    
    best_metrics = comparison.results[best_model_name]
    model_id = best_metrics.model_id
    
    print("\n" + "="*80)
    print(f"TOP-K ANALYSIS FOR BEST MODEL: {best_model_name}")
    print("="*80)
    print(f"Model ID: {model_id}")
    print(f"Current MRR: {best_metrics.mrr:.4f}")
    print(f"Testing K values: {k_values}")
    
    # Step 2.2: Load model and encode chunks
    print("\n📥 Loading model...")
    model = SentenceTransformer(model_id)
    
    print("📊 Encoding chunks...")
    chunk_texts = [f"{c['context']}\n{c['text']}" for c in chunks]
    chunk_embeddings = model.encode(
        chunk_texts,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    # Step 2.3: Test each k value
    print(f"\n🔍 Testing {len(comparison.test_queries)} queries at different K values...")
    
    recall_at_k = {k: [] for k in k_values}
    precision_at_k = {k: [] for k in k_values}
    
    for query_data in comparison.test_queries:
        query = query_data['query']
        relevant_ids = set(query_data['relevant_ids'])
        
        # Encode query
        query_embedding = model.encode([query], convert_to_numpy=True)
        
        # Get similarities and sort
        similarities = util.cos_sim(query_embedding, chunk_embeddings)[0]
        top_indices = np.argsort(-similarities.numpy())
        
        # Get all chunk IDs in order
        all_chunk_ids = [
            chunks[idx].get('metadata', {}).get('id', '')
            for idx in top_indices
        ]
        
        # Calculate metrics at each k
        for k in k_values:
            top_k_ids = set(all_chunk_ids[:k])
            
            # Recall: What % of relevant docs found?
            recall = len(top_k_ids & relevant_ids) / len(relevant_ids) if relevant_ids else 0
            recall_at_k[k].append(recall)
            
            # Precision: What % of retrieved docs relevant?
            precision = len(top_k_ids & relevant_ids) / k if k > 0 else 0
            precision_at_k[k].append(precision)
    
    # Step 2.4: Calculate averages
    avg_recall = {k: np.mean(scores) for k, scores in recall_at_k.items()}
    avg_precision = {k: np.mean(scores) for k, scores in precision_at_k.items()}
    
    # Step 2.5: Print results table
    print("\n" + "="*80)
    print("RESULTS: RECALL & PRECISION AT DIFFERENT K VALUES")
    print("="*80)
    print(f"{'K':<6} {'Recall':<12} {'Precision':<12} {'Gain from Prev':<18} {'Cost':<10}")
    print("-"*80)
    
    prev_recall = 0
    for i, k in enumerate(k_values):
        recall = avg_recall[k]
        precision = avg_precision[k]
        gain = recall - prev_recall
        extra_chunks = k - (k_values[i-1] if i > 0 else 0)
        
        # Format with color coding
        gain_str = f"+{gain:.4f} ({gain*100:+.1f}%)"
        cost_str = f"+{extra_chunks} chunks" if i > 0 else f"{k} chunks"
        
        print(f"{k:<6} {recall:<12.4f} {precision:<12.4f} {gain_str:<18} {cost_str:<10}")
        prev_recall = recall
    
    print("="*80)
    
    # Step 2.6: Analyze diminishing returns
    print("\n💡 DIMINISHING RETURNS ANALYSIS")
    print("-"*80)
    
    for i in range(1, len(k_values)):
        k_prev = k_values[i-1]
        k_curr = k_values[i]
        
        gain = avg_recall[k_curr] - avg_recall[k_prev]
        extra_chunks = k_curr - k_prev
        gain_per_chunk = gain / extra_chunks
        
        # Color code by efficiency
        if gain > 0.05:
            status = "✅ GOOD GAIN"
        elif gain > 0.02:
            status = "⚠️  MODERATE GAIN"
        else:
            status = "❌ DIMINISHING RETURNS"
        
        print(f"K={k_prev}→{k_curr}: "
              f"Gain: {gain:.4f} (+{gain*100:.1f}%) | "
              f"Cost: +{extra_chunks} chunks | "
              f"Efficiency: {gain_per_chunk:.4f}/chunk | "
              f"{status}")
    
    # Step 2.7: Make recommendation
    print("\n" + "="*80)
    print("RECOMMENDATION")
    print("="*80)
    
    # Find optimal k (where we hit 90% recall or diminishing returns kick in)
    optimal_k = k_values[0]
    
    # Strategy 1: First k that achieves 90% recall
    for k in k_values:
        if avg_recall[k] >= 0.90:
            optimal_k = k
            break
    
    # Strategy 2: If no 90% recall, use diminishing returns
    if avg_recall[optimal_k] < 0.90:
        for i in range(1, len(k_values)):
            k_curr = k_values[i]
            k_prev = k_values[i-1]
            gain = avg_recall[k_curr] - avg_recall[k_prev]
            
            if gain < 0.03:  # Less than 3% gain
                optimal_k = k_prev
                break
    
    print(f"\n🎯 OPTIMAL K: {optimal_k}")
    print(f"\n📊 Performance at K={optimal_k}:")
    print(f"   • Recall: {avg_recall[optimal_k]:.4f} ({avg_recall[optimal_k]*100:.1f}%)")
    print(f"   • Precision: {avg_precision[optimal_k]:.4f} ({avg_precision[optimal_k]*100:.1f}%)")
    print(f"   • Retrieves {optimal_k} chunks per query")
    
    print(f"\n💭 Why K={optimal_k}?")
    if avg_recall[optimal_k] >= 0.90:
        print(f"   ✅ Achieves 90%+ recall ({avg_recall[optimal_k]*100:.1f}%)")
    
    # Check next k
    next_k_idx = k_values.index(optimal_k) + 1
    if next_k_idx < len(k_values):
        next_k = k_values[next_k_idx]
        gain_to_next = avg_recall[next_k] - avg_recall[optimal_k]
        extra_chunks = next_k - optimal_k
        print(f"   📉 Going to K={next_k} only gains {gain_to_next*100:.1f}% "
              f"for {extra_chunks} more chunks (not worth it)")
    
    print(f"\n💡 UPDATE YOUR RAG CONFIG:")
    print(f"   config.top_k = {optimal_k}")
    
    # Step 2.8: Plot results
    plot_topk_results(k_values, avg_recall, avg_precision, 
                     best_model_name, optimal_k)
    
    # Step 2.9: Save results
    results = {
        'model_name': best_model_name,
        'model_id': model_id,
        'optimal_k': optimal_k,
        'k_values': k_values,
        'recall_at_k': avg_recall,
        'precision_at_k': avg_precision
    }
    
    import json
    with open('optimal_topk_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\n💾 Results saved to: optimal_topk_results.json")
    
    return results


def plot_topk_results(k_values, recall, precision, model_name, optimal_k):
    """Plot recall and precision curves with optimal k marked"""
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    # Plot recall
    recall_values = [recall[k] for k in k_values]
    ax.plot(k_values, recall_values, marker='o', linewidth=2.5, 
            markersize=10, label='Recall', color='#2E86AB')
    
    # Plot precision
    precision_values = [precision[k] for k in k_values]
    ax.plot(k_values, precision_values, marker='s', linewidth=2.5,
            markersize=10, label='Precision', color='#A23B72')
    
    # Mark optimal k
    ax.axvline(x=optimal_k, color='green', linestyle='--', 
               linewidth=2, label=f'Optimal K={optimal_k}')
    ax.axhline(y=0.90, color='red', linestyle=':', 
               linewidth=1.5, alpha=0.7, label='90% Target')
    
    # Add value labels
    for k, r in zip(k_values, recall_values):
        ax.annotate(f'{r:.3f}', xy=(k, r), xytext=(0, 10),
                   textcoords='offset points', ha='center', 
                   fontsize=9, fontweight='bold')
    
    ax.set_xlabel('K (Number of Retrieved Chunks)', fontsize=13, fontweight='bold')
    ax.set_ylabel('Score', fontsize=13, fontweight='bold')
    ax.set_title(f'Top-K Analysis: {model_name}', fontsize=15, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(fontsize=11, loc='lower right')
    ax.set_ylim([0, 1.05])
    ax.set_xticks(k_values)
    
    plt.tight_layout()
    filename = f'topk_analysis_{model_name}.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"\n📊 Plot saved to: {filename}")
    plt.show()


# ============================================================================
# STEP 3: RUN THE TOP-K ANALYSIS (ADD THIS TO YOUR CODE)
# ============================================================================

# After you run your model comparison:
# comparison, results_df = run_model_comparison(chunks, save_best=True)

# NOW ADD THIS:
print("\n" + "="*80)
print("STEP 2: ANALYZING OPTIMAL TOP-K FOR BEST MODEL")
print("="*80)

# Run top-k analysis on best model
topk_results = analyze_optimal_topk_for_best_model(
    comparison=comparison,
    chunks=chunks,
    best_model_name=None,  # Auto-select best model
    k_values=[1, 2, 3, 5, 7, 10, 15]  # Test these k values
)

print(f"\n✅ ANALYSIS COMPLETE!")
print(f"🎯 Optimal K for your RAG system: {topk_results['optimal_k']}")
print(f"📊 Recall at K={topk_results['optimal_k']}: "
      f"{topk_results['recall_at_k'][topk_results['optimal_k']]:.4f}")


# ============================================================================
# COMPLETE EXAMPLE: Full Pipeline
# ============================================================================

def complete_embedding_optimization_pipeline(chunks: List[Dict]):
    """
    Complete pipeline: Compare models → Find best → Optimize top-k
    """
    print("\n" + "🚀"*40)
    print("COMPLETE EMBEDDING OPTIMIZATION PIPELINE")
    print("🚀"*40)
    
    # Step 1: Compare embedding models
    print("\n📊 STEP 1: Comparing embedding models...")
    comparison, results_df = run_model_comparison(chunks, save_best=True)
    
    # Step 2: Find optimal top-k for best model
    print("\n📊 STEP 2: Finding optimal top-k...")
    topk_results = analyze_optimal_topk_for_best_model(
        comparison=comparison,
        chunks=chunks,
        k_values=[1, 2, 3, 5, 7, 10, 15]
    )
    
    # Step 3: Print final recommendations
    print("\n" + "="*80)
    print("FINAL RECOMMENDATIONS FOR YOUR RAG SYSTEM")
    print("="*80)
    
    best_model = topk_results['model_name']
    optimal_k = topk_results['optimal_k']
    
    print(f"\n🏆 Best Embedding Model: {best_model}")
    print(f"   Model ID: {topk_results['model_id']}")
    
    print(f"\n🎯 Optimal Top-K: {optimal_k}")
    print(f"   Recall: {topk_results['recall_at_k'][optimal_k]:.4f}")
    print(f"   Precision: {topk_results['precision_at_k'][optimal_k]:.4f}")
    
    print(f"\n📝 UPDATE YOUR CONFIG:")
    print(f"   class RAGConfig:")
    print(f"       base_model = '{topk_results['model_id']}'")
    print(f"       top_k = {optimal_k}")
    
    print(f"\n💾 Files Generated:")
    print(f"   • models/best/embedding_model/")
    print(f"   • optimal_topk_results.json")
    print(f"   • topk_analysis_{best_model}.png")
    print(f"   • embedding_comparison_results.json")
    
    return {
        'best_model': best_model,
        'optimal_k': optimal_k,
        'comparison': comparison,
        'topk_results': topk_results
    }


# ============================================================================
# HOW TO USE IN YOUR CODE
# ============================================================================

if __name__ == "__main__":
    # Load your chunks
    # ✅ ADD THIS:
    topk_results = analyze_optimal_topk_for_best_model(
        comparison=comparison,
        chunks=chunks,
        k_values=[1, 2, 3, 5, 7, 10, 15]
    )

print(f"🎯 Optimal K: {topk_results['optimal_k']}")